In [15]:
# import libraries
import torch
import spacy
import random
from nltk.translate.bleu_score import corpus_bleu
from nltk.metrics import *

random.seed(0)

In [16]:
# read data
# run once
# with open('data/train.txt','r') as source:
#     data = [ (random.random(), line) for line in source ]
# data.sort()
# data = [line for _, line in data]

# data_size = len(data)
# train_data = data[:int(data_size * 0.9)]
# val_data = data[int(data_size * 0.9):]

# with open('data/test.txt','r') as source:
#     test_data = source.readlines()

In [17]:
# preprocess data into train, val, and test set
# run once
# for src_path, tgt_path, dataset in zip(['src_train.txt', 'src_val.txt', 'src_test.txt'], \
#                                        ['tgt_train.txt', 'tgt_val.txt', 'tgt_test.txt'], \
#                                        [train_data, val_data, test_data]):
#     with open('data/' + src_path,'w+') as src:
#         with open('data/' + tgt_path,'w+') as tgt:
#             for line in dataset:
#                 cur_file = src
#                 for tok in line.strip().split(sep=' '):
#                     if tok == 'IN:':
#                         continue
#                     elif tok == "OUT:":
#                         cur_file.write('\n')
#                         cur_file = tgt
#                     else:
#                         cur_file.write(tok+' ')
#                 cur_file.write('\n')

In [18]:
# evaluation methods
def eval(path_to_gold, path_to_pred):
    candidates = []
    references = []

    with open(path_to_gold, 'r') as gold_f:
        gold = gold_f.readlines()
        gold_size = len(gold)
        with open(path_to_pred, 'r') as pred_f:
            pred = pred_f.readlines()
            pred_size = len(pred)
            if gold_size != pred_size:
                Exception()
            sent_tp = 0
            c_precison = 0
            c_recall = 0
            c_f = 0
            tp = 0
            fp = 0
            tn = 0
            fn = 0
            for i, gold_line in enumerate(gold):
                pred_line = pred[i].strip()
                gold_line = gold_line.strip()
                pred_set = set(pred_line.split())
                gold_set = set(gold_line.split())
                candidates.append(pred_line)
                references.append([gold_line])
                if 'I_JUMP' in gold_set:
                    if 'I_JUMP' in pred_set:
                        tp += 1
                    else:
                        fn += 1
                else:
                    if 'I_JUMP' in pred_set:
                        fp += 1
                    else:
                        tn += 1

                if precision(pred_set, gold_set):
                    c_precison += precision(pred_set, gold_set)
                if recall(pred_set, gold_set):
                    c_recall += recall(pred_set, gold_set)
                if f_measure(pred_set, gold_set):
                    c_f += f_measure(pred_set, gold_set)
                if gold_line == pred_line:
                    sent_tp += 1

    if tp+fp:
        p = tp / (tp+fp)
    else:
        p = 0.0
    if tp+fn:
        r = tp / (tp+fn)
    else:
        r = 0.0
    if p+r:
        f1 = 2*r*p/(r+p)
    else:
        f1 = 0.0

    print('Sent-Level Acc', sent_tp/gold_size)
    print('Average Precision', c_precison/gold_size)
    print('Average Recall', c_recall/gold_size)
    print('Average F1', c_f/gold_size)
    print('New Primitive Precision', p)
    print('New Primitive Recall', r)
    print('New Primitive F1', f1)
    print('BLEU', corpus_bleu(references, candidates))

In [19]:
# build baseline vocab (before data augmentation)
# run once
# !onmt_build_vocab -config vocab_base.yaml -n_sample -1

In [20]:
# baseline training
# run once
# !onmt_train -config train_base.yaml

In [21]:
# baseline prediction
# run once
# !onmt_translate -model models/base.pt -src data/src_test.txt -output pred/base.txt -gpu 0

In [22]:
# evaluate baseline model
# eval('data/tgt_test.txt', 'pred/base.txt')

In [23]:
# with open('data/src_train.txt','r') as f:
#     src_train = f.readlines()
# with open('data/tgt_train.txt','r') as f:
#     tgt_train = f.readlines()

In [24]:
# 1-replace data augmentation
# run once
# with open('data/src_train_aug_1.txt', 'w+') as src:
#     with open('data/tgt_train_aug_1.txt','w+') as tgt:
#         for src_1, tgt_1 in list(zip(src_train, tgt_train)):
#             if len(src_1.split()) > 1:
#                 src_1_aug = src_1.replace('run','jump')
#                 tgt_1_aug = tgt_1.replace('I_RUN','I_JUMP')
#                 src.write(f'{src_1_aug}')
#                 tgt.write(f'{tgt_1_aug}')

In [25]:
# build vocab (use 1-replace data augmentation method)
# run once
# !onmt_build_vocab -config vocab_aug_1.yaml -n_sample -1

In [26]:
# aug 1 training
# run once
# !onmt_train -config train_aug_1.yaml

In [27]:
# aug 1 prediction
# run once
# !onmt_translate -model models/aug1.pt -src data/src_test.txt -output pred/aug1.txt -gpu 0

In [28]:
# evaluate aug 1 model
# eval('data/tgt_test.txt', 'pred/aug1.txt')

In [29]:
# nlp = spacy.load("en_core_web_trf")

In [30]:
# threshold = 0

In [31]:
# run once
# with open('data/src_train_aug_pr.txt', 'w+') as src:
#     with open('data/tgt_train_aug_pr.txt', 'w+') as tgt:
#         for index, (src_1, tgt_1) in enumerate(list(zip(src_train, tgt_train))[0:-10]):
#             for src_2, tgt_2 in list(zip(src_train, tgt_train))[index + 1: index + 11]:
#                 if recall(set(src_1.split()), set(src_2.split())) <= threshold:
#                     src_1_doc = nlp(src_1.strip())
#                     src_2_doc = nlp(src_2.strip())
#                     tgt_1_doc = nlp(tgt_1.strip())
#                     tgt_2_doc = nlp(tgt_2.strip())
#                     for token_1 in src_1_doc:
#                         # print(token.text, token.pos_, token.tag_, token.dep_, [t.text for t in token.subtree])
#                         # if ([t.text for t in token.subtree] != [t.text for t in src_1_doc]) or \
#                         # (len([t.text for t in token.subtree]) == 1):
#                         #     pass
#                         for token_2 in src_2_doc:
#                             # print(token.text, token.pos_, token.tag_, token.dep_, [t.text for t in token.subtree])
#                             # if ([t.text for t in token.subtree] != [t.text for t in src_2_doc]) or \
#                             # (len([t.text for t in token.subtree]) == 1):
#                             #     pass
#                             if (token_1.pos_ == token_2.pos_) and (token_1.dep_ == token_2.dep_):
#                                 src.write(f'{src_1.replace(token_1.text, token_2.text)}')
#                                 break
#                         else:
#                             continue
#                         break
#                     src.write(f'\n')
#                     for token_1 in tgt_1_doc:
#                         # print(token.text, token.pos_, token.tag_, token.dep_, [t.text for t in token.subtree])
#                         # if ([t.text for t in token.subtree] != [t.text for t in tgt_1_doc]) or \
#                         # (len([t.text for t in token.subtree]) == 1):
#                         #     pass
#                         for token_2 in tgt_2_doc:
#                             # print(token.text, token.pos_, token.tag_, token.dep_, [t.text for t in token.subtree])
#                             # if ([t.text for t in token.subtree] != [t.text for t in tgt_2_doc]) or \
#                             # (len([t.text for t in token.subtree]) == 1):
#                             #     pass
#                             if (token_1.pos_ == token_2.pos_) and (token_1.dep_ == token_2.dep_):
#                                 tgt.write(f'{tgt_1.replace(token_1.text, token_2.text)}')
#                                 break
#                         else:
#                             continue
#                         break
#                     tgt.write(f'\n')

In [32]:
# run once
# with open('data/src_train_aug_p.txt', 'w+') as src:
#     with open('data/tgt_train_aug_p.txt', 'w+') as tgt:
#         for index, (src_1, tgt_1) in enumerate(list(zip(src_train, tgt_train))[0:-10]):
#             for src_2, tgt_2 in list(zip(src_train, tgt_train))[index + 1: index + 11]:
#                 if recall(set(src_1.split()), set(src_2.split())) <= threshold:
#                     src_1_doc = nlp(src_1.strip())
#                     src_2_doc = nlp(src_2.strip())
#                     tgt_1_doc = nlp(tgt_1.strip())
#                     tgt_2_doc = nlp(tgt_2.strip())
#                     for token_1 in src_1_doc:
#                         # print(token.text, token.pos_, token.tag_, token.dep_, [t.text for t in token.subtree])
#                         # if ([t.text for t in token.subtree] != [t.text for t in src_1_doc]) or \
#                         # (len([t.text for t in token.subtree]) == 1):
#                         #     pass
#                         for token_2 in src_2_doc:
#                             # print(token.text, token.pos_, token.tag_, token.dep_, [t.text for t in token.subtree])
#                             # if ([t.text for t in token.subtree] != [t.text for t in src_2_doc]) or \
#                             # (len([t.text for t in token.subtree]) == 1):
#                             #     pass
#                             if token_1.pos_ == token_2.pos_:
#                                 src.write(f'{src_1.replace(token_1.text, token_2.text)}')
#                                 break
#                         else:
#                             continue
#                         break
#                     src.write(f'\n')
#                     for token_1 in tgt_1_doc:
#                         # print(token.text, token.pos_, token.tag_, token.dep_, [t.text for t in token.subtree])
#                         # if ([t.text for t in token.subtree] != [t.text for t in tgt_1_doc]) or \
#                         # (len([t.text for t in token.subtree]) == 1):
#                         #     pass
#                         for token_2 in tgt_2_doc:
#                             # print(token.text, token.pos_, token.tag_, token.dep_, [t.text for t in token.subtree])
#                             # if ([t.text for t in token.subtree] != [t.text for t in tgt_2_doc]) or \
#                             # (len([t.text for t in token.subtree]) == 1):
#                             #     pass
#                             if token_1.pos_ == token_2.pos_:
#                                 tgt.write(f'{tgt_1.replace(token_1.text, token_2.text)}')
#                                 break
#                         else:
#                             continue
#                         break
#                     tgt.write(f'\n')

In [33]:
# run once
# with open('data/src_train_aug_r.txt', 'w+') as src:
#     with open('data/tgt_train_aug_r.txt', 'w+') as tgt:
#         for index, (src_1, tgt_1) in enumerate(list(zip(src_train, tgt_train))[0:-10]):
#             for src_2, tgt_2 in list(zip(src_train, tgt_train))[index + 1: index + 11]:
#                 if recall(set(src_1.split()), set(src_2.split())) <= threshold:
#                     src_1_doc = nlp(src_1.strip())
#                     src_2_doc = nlp(src_2.strip())
#                     tgt_1_doc = nlp(tgt_1.strip())
#                     tgt_2_doc = nlp(tgt_2.strip())
#                     for token_1 in src_1_doc:
#                         # print(token.text, token.pos_, token.tag_, token.dep_, [t.text for t in token.subtree])
#                         # if ([t.text for t in token.subtree] != [t.text for t in src_1_doc]) or \
#                         # (len([t.text for t in token.subtree]) == 1):
#                         #     pass
#                         for token_2 in src_2_doc:
#                             # print(token.text, token.pos_, token.tag_, token.dep_, [t.text for t in token.subtree])
#                             # if ([t.text for t in token.subtree] != [t.text for t in src_2_doc]) or \
#                             # (len([t.text for t in token.subtree]) == 1):
#                             #     pass
#                             if token_1.dep_ == token_2.dep_:
#                                 src.write(f'{src_1.replace(token_1.text, token_2.text)}')
#                                 break
#                         else:
#                             continue
#                         break
#                     src.write(f'\n')
#                     for token_1 in tgt_1_doc:
#                         # print(token.text, token.pos_, token.tag_, token.dep_, [t.text for t in token.subtree])
#                         # if ([t.text for t in token.subtree] != [t.text for t in tgt_1_doc]) or \
#                         # (len([t.text for t in token.subtree]) == 1):
#                         #     pass
#                         for token_2 in tgt_2_doc:
#                             # print(token.text, token.pos_, token.tag_, token.dep_, [t.text for t in token.subtree])
#                             # if ([t.text for t in token.subtree] != [t.text for t in tgt_2_doc]) or \
#                             # (len([t.text for t in token.subtree]) == 1):
#                             #     pass
#                             if token_1.dep_ == token_2.dep_:
#                                 tgt.write(f'{tgt_1.replace(token_1.text, token_2.text)}')
#                                 break
#                         else:
#                             continue
#                         break
#                     tgt.write(f'\n')

In [34]:
# build vocab (use PR data augmentation method)
# run once
# !onmt_build_vocab -config vocab_aug_pr.yaml -n_sample -1

In [35]:
# aug pr training
# run once
# !onmt_train -config train_aug_pr.yaml -skip_empty_level silent

In [36]:
# build vocab (use P data augmentation method)
# run once
# !onmt_build_vocab -config vocab_aug_p.yaml -n_sample -1

In [37]:
# aug p training
# run once
# !onmt_train -config train_aug_p.yaml -skip_empty_level silent

In [38]:
# build vocab (use R data augmentation method)
# run once
# !onmt_build_vocab -config vocab_aug_r.yaml -n_sample -1

In [39]:
# aug r training
# run once
# !onmt_train -config train_aug_r.yaml -skip_empty_level silent

In [40]:
# prediction
# run once
# !onmt_translate -model models/augpr.pt -src data/src_test.txt -output pred/augpr.txt -gpu 0
# !onmt_translate -model models/augp.pt -src data/src_test.txt -output pred/augp.txt -gpu 0
# !onmt_translate -model models/augr.pt -src data/src_test.txt -output pred/augr.txt -gpu 0

In [41]:
eval('data/tgt_test.txt', 'pred/augpr.txt')

Sent-Level Acc 0.0
Average Precision 0.6349381434380154
Average Recall 0.7289644432909537
Average F1 0.6711579103481714
New Primitive Precision 1.0
New Primitive Recall 0.007915909680768232
New Primitive F1 0.015707480365649545
BLEU 0.7197529308967632


In [42]:
eval('data/tgt_test.txt', 'pred/augp.txt')

Sent-Level Acc 0.007267064624967558
Average Precision 0.5020979323470771
Average Recall 0.651215503071202
Average F1 0.5499195638113354
New Primitive Precision 1.0
New Primitive Recall 0.20776018686737607
New Primitive F1 0.34404211883528524
BLEU 0.42789970728204174


In [43]:
eval('data/tgt_test.txt', 'pred/augr.txt')

Sent-Level Acc 0.059563976122501944
Average Precision 0.6745933904317084
Average Recall 0.8066549874556828
Average F1 0.7262899657657345
New Primitive Precision 1.0
New Primitive Recall 0.5960290682584999
New Primitive F1 0.7468899910561835
BLEU 0.6517496994968263


In [44]:
eval('data/tgt_test.txt', 'pred/aug1.txt')

Sent-Level Acc 0.9962366986763561
Average Precision 0.9998702309888399
Average Recall 1.0
Average F1 0.9999134873258932
New Primitive Precision 1.0
New Primitive Recall 0.9997404619776797
New Primitive F1 0.999870214146658
BLEU 0.9996333754237531


In [45]:
eval('data/tgt_test.txt', 'pred/base.txt')

Sent-Level Acc 0.0010381520892810796
Average Precision 0.6536248810450793
Average Recall 0.9644432909421262
Average F1 0.7742352897557331
New Primitive Precision 1.0
New Primitive Recall 0.0028549182455229693
New Primitive F1 0.005693581780538303
BLEU 0.7914298126967355


In [47]:
def count(path_to_file, keyword):
    with open(path_to_file, 'r') as f:
        tokens = []
        for line in f.readlines():
            tokens += line.split()
        print(tokens.count(keyword))

In [52]:
for keyword in ['I_JUMP', 'I_WALK', 'I_RUN', 'I_LOOK']:
    for path in ['pred/base.txt', 'pred/aug1.txt', 'pred/augpr.txt', 'pred/augp.txt', 'pred/augr.txt']:
        print(path, keyword)
        count(path, keyword)

pred/base.txt I_JUMP
22
pred/aug1.txt I_JUMP
31883
pred/augpr.txt I_JUMP
129
pred/augp.txt I_JUMP
9224
pred/augr.txt I_JUMP
27828
pred/base.txt I_WALK
6924
pred/aug1.txt I_WALK
6543
pred/augpr.txt I_WALK
35449
pred/augp.txt I_WALK
7744
pred/augr.txt I_WALK
11834
pred/base.txt I_RUN
6640
pred/aug1.txt I_RUN
6584
pred/augpr.txt I_RUN
11377
pred/augp.txt I_RUN
6336
pred/augr.txt I_RUN
15182
pred/base.txt I_LOOK
6636
pred/aug1.txt I_LOOK
6553
pred/augpr.txt I_LOOK
10431
pred/augp.txt I_LOOK
7134
pred/augr.txt I_LOOK
12599
